# Codigo propio

In [1]:
import lib.custom_metrics as metrics
import lib.evaluation as ev
import lib.plotting as plot
import lib.models as models
import lib.dataset as dt

import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

from importlib import reload

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
DR_LEVELS_PER_CLASS = [[0], [1,2,3,4]]

IMAGE_SIZE = (540, 540, 3)

# Specify dataset files
TRAIN_FILE = 'DATASET-TRAIN-80.csv'
VALIDATION_FILE = 'DATASET-VALIDATION-10.csv'

TRAINING_BATCH_SIZE = 4
TRAINING_DATA_AUG = True
TRAINING_BALANCED = False
TRAINING_SHUFFLE = True
TRAINING_PREFETCH = 1
TRAINING_TAKE_SIZE = None

VALIDATION_BATCH_SIZE = 4
VALIDATION_DATA_AUG = False
VALIDATION_BALANCED = False
VALIDATION_SHUFFLE = False
VALIDATION_PREFETCH = 1
VALIDATION_TAKE_SIZE = None

In [4]:
reload(dt)

'''
def create_dataset_new(csv_file, 
                        list_list_classes,
                        balanced=False,
                        apply_data_augmentation=False,
                        batch_size=1, 
                        prefetch_buffer=None, 
                        shuffle=False,
                        size=None):
'''

train_dataset, y_true_train = dt.create_dataset_new(TRAIN_FILE, 
                                                    DR_LEVELS_PER_CLASS, 
                                                    balanced=TRAINING_BALANCED, 
                                                    apply_data_augmentation=TRAINING_DATA_AUG, 
                                                    batch_size=TRAINING_BATCH_SIZE, 
                                                    prefetch_buffer=TRAINING_PREFETCH, 
                                                    shuffle=TRAINING_SHUFFLE,
                                                    size=TRAINING_TAKE_SIZE)

val_dataset, y_true_val = dt.create_dataset_new(VALIDATION_FILE, 
                                                DR_LEVELS_PER_CLASS, 
                                                balanced=VALIDATION_BALANCED,
                                                apply_data_augmentation=VALIDATION_DATA_AUG,
                                                batch_size=VALIDATION_BATCH_SIZE,
                                                prefetch_buffer=VALIDATION_PREFETCH, 
                                                shuffle=VALIDATION_SHUFFLE, 
                                                size=VALIDATION_TAKE_SIZE)

Se aplica shuffle
Se aplica data aug


In [5]:
# Define model
reload(models)

# model = models.mobileNetV2.get_model(input_shape=(540,540,3), num_outputs=3)
model = models.efficientNetB2.get_model(input_shape=(540,540,3), num_outputs=len(DR_LEVELS_PER_CLASS))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb2 (Functional)  (None, 1408)              7768569   
_________________________________________________________________
flatten (Flatten)            (None, 1408)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2818      
Total params: 7,771,387
Trainable params: 7,703,812
Non-trainable params: 67,575
_________________________________________________________________


In [6]:
# Load weights if they were saved
base_path = 'saved_weights/efficientNetB2/SGD/'
save_path = base_path + 'model.h5'

if not os.path.exists(base_path):
    os.mkdir(base_path)

if os.path.exists(save_path):
    model.load_weights(save_path)
    print('Model loaded')

In [7]:
# Save ground truth for validation dataset
np.save(base_path + 'ground_truth_val.npy', y_true_val)

In [8]:
# Compile model
reload(metrics)

classes_names = [''.join(list(map(str,class_))) for class_ in DR_LEVELS_PER_CLASS]

print(classes_names)

metric_AUC_0_1234 = metrics.Wrapper_AUC(classes=[1], original_dr_lvls=classes_names)

metric_Sp_at_95_Sens_0_1234 = metrics.Wrapper_SpecificityAtSensitivity(sensitivity=0.95, classes=[1], original_dr_lvls=classes_names)

['0', '1234']


In [9]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

# Create callbacks
tensorboard_callback = tf.keras.callbacks.TensorBoard('logs/efficientNetB2_EYEPACS', histogram_freq=1, write_graph=False)

cbacks = [tf.keras.callbacks.ModelCheckpoint(base_path + 'best' + metric_AUC_0_1234.get_custom_name() + '.h5', 
                                             monitor='val_' + metric_AUC_0_1234.get_custom_name(),
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),

          tf.keras.callbacks.ModelCheckpoint(base_path + 'best' + metric_Sp_at_95_Sens_0_1234.get_custom_name() + '.h5', 
                                             monitor='val_' + metric_Sp_at_95_Sens_0_1234.get_custom_name(),
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),

          tf.keras.callbacks.ModelCheckpoint(base_path + 'best_loss.h5',
                                             monitor='val_loss',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='min'),
          tensorboard_callback]

In [10]:
# See Tensorboard
%tensorboard --logdir logs

In [11]:
validation_dir = base_path+'validation_outputs/'

if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, clipnorm=1.0), 
              metrics=[metric_AUC_0_1234, # DR levels: 0 vs 1,2,3,4
                       metric_Sp_at_95_Sens_0_1234, # DR levels: 0 vs 1,2,3,4
                       #metrics.RunningValidation(path=validation_dir) # THIS METRIC MUST BE INIZIALIZATED BEFORE EVERY TRAINING
                      ]) 

num_epochs = 2000

history = model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, verbose=1, callbacks=cbacks)

Epoch 1/2000
    1/14206 [..............................] - ETA: 7s - loss: 0.7278 - AUC_DRlvls_0_1234: 0.5000 - Sp_at_95_sens_DRlvls_0_1234: 0.5000WARNING:tensorflow:From /home/alumno/miguel_herrera/lib/python3.8/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
14206/14206 [==============================] - 3641s 256ms/step - loss: 0.5145 - AUC_DRlvls_0_1234: 0.7191 - Sp_at_95_sens_DRlvls_0_1234: 0.1196 - val_loss: 1.1930 - val_AUC_DRlvls_0_1234: 0.5332 - val_Sp_at_95_sens_DRlvls_0_1234: 0.0592
Epoch 2/2000
14206/14206 [==============================] - 3651s 257ms/step - loss: 0.4551 - AUC_DRlvls_0_1234: 0.7809 - Sp_at_95_sens_DRlvls_0_1234: 0.1708 - val_loss: 1.4464 - val_AUC_DRlvls_0_1234: 0.4820 - val_Sp_at_95_sens_DRlvls_0_1234: 0.0368
Epoch 3/2000
14206/14206 [==============================] - 3629s 25

KeyboardInterrupt: 